In [1]:
!pip install --upgrade torch torchvision

In [5]:
# Bagian 1: Import Library dan Persiapan Data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Gunakan GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Prepare the data
transform = transforms.Compose([
    transforms.Resize((14, 14)),  # Kurangi resolusi gambar untuk mempercepat
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Kurangi batch size untuk menghemat memori
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [6]:
# Bagian 2: Definisi Model CNN
def create_cnn(kernel_size, pooling_type):
    pooling = nn.MaxPool2d if pooling_type == "max" else nn.AvgPool2d

    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size, stride=1, padding=kernel_size // 2)
            self.pool = pooling(2, 2)
            self.conv2 = nn.Conv2d(16, 32, kernel_size=kernel_size, stride=1, padding=kernel_size // 2)
            self.fc1 = nn.Linear(32 * 3 * 3, 128)  # Disesuaikan dengan output size resolusi input
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            x = x.view(-1, 32 * 3 * 3)
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    return CNN().to(device)

In [7]:
# Bagian 3: Fungsi Pelatihan Model dengan Early Stopping
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.best_val_loss = float('inf')
        self.counter = 0
        self.stop_training = False

    def __call__(self, val_loss):
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop_training = True

def train_model(model, optimizer_name, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    early_stopping = EarlyStopping(patience=3)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        early_stopping(val_loss)
        if early_stopping.stop_training:
            print("Early stopping triggered")
            break

        scheduler.step()

In [8]:
# Bagian 4: Konfigurasi Eksperimen
kernel_sizes = [3, 5, 7]
pooling_types = ["max", "avg"]
optimizers = ["SGD", "RMSprop", "Adam"]
epochs_list = [5, 50, 100]  # Kurangi jumlah epoch untuk eksperimen awal


In [ ]:
# Bagian 5: Menjalankan Eksperimen
results = {}
for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer_name in optimizers:
            for epochs in epochs_list:
                print(f"\nTraining with Kernel: {kernel_size}, Pooling: {pooling_type}, Optimizer: {optimizer_name}, Epochs: {epochs}")
                model = create_cnn(kernel_size, pooling_type)
                train_model(model, optimizer_name, epochs)


Training with Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5, Train Loss: 3421.2891, Val Loss: 2.2472, Val Acc: 0.3362
Epoch 2/5, Train Loss: 3250.8258, Val Loss: 2.0305, Val Acc: 0.4086
Epoch 3/5, Train Loss: 2584.3416, Val Loss: 1.3914, Val Acc: 0.5717
Epoch 4/5, Train Loss: 1747.3304, Val Loss: 0.9974, Val Acc: 0.6498
Epoch 5/5, Train Loss: 1367.0385, Val Loss: 0.8430, Val Acc: 0.7097

Training with Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 50
Epoch 1/50, Train Loss: 3424.7549, Val Loss: 2.2670, Val Acc: 0.2388
Epoch 2/50, Train Loss: 3351.4796, Val Loss: 2.1864, Val Acc: 0.3497
Epoch 3/50, Train Loss: 3071.3589, Val Loss: 1.8300, Val Acc: 0.4065
Epoch 4/50, Train Loss: 2277.6749, Val Loss: 1.2627, Val Acc: 0.6131
Epoch 5/50, Train Loss: 1661.9026, Val Loss: 0.9860, Val Acc: 0.6730
Epoch 6/50, Train Loss: 1373.2842, Val Loss: 0.8568, Val Acc: 0.6927
Epoch 7/50, Train Loss: 1239.0582, Val Loss: 0.7951, Val Acc: 0.7145
Epoch 8/50, Train Loss: 1167.1124, Val Loss: